In [1]:
import azureml.core

from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Workspace, Environment, Datastore, Experiment
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.runconfig import RunConfiguration

from azureml.exceptions import WebserviceException
from azureml.data.data_reference import DataReference

from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import PipelineData, Pipeline
import json
import os


In [2]:
with open('config.json', 'r') as jsonfile:
    ws_config = json.load(jsonfile)

interactive_auth = InteractiveLoginAuthentication(tenant_id=ws_config['tenantId'])

ws = Workspace(
    subscription_id=ws_config['subscription_id'],
    resource_group=ws_config['resource_group'],
    workspace_name=ws_config['workspace_name'],
    auth=interactive_auth,
)

In [3]:
blob_datastore_name='shiftdatastore' # Name of the datastore to workspace
container_name=os.getenv("BLOB_CONTAINER", "news20container") # Name of Azure blob container
account_name=os.getenv("BLOB_ACCOUNTNAME", "shiftreference") # Storage account name
account_key=os.getenv("AZURE_STORAGE_KEY") # Storage account key

try:
    datastore = Datastore.get(ws, blob_datastore_name)
except:
    datastore = Datastore.register_azure_blob_container(workspace=ws, 
                                                             datastore_name=blob_datastore_name, 
                                                             container_name=container_name, 
                                                             account_name=account_name,
                                                             account_key=account_key)


In [4]:
blob_input_data = DataReference(
    datastore,
    data_reference_name="rawdata",
    path_on_datastore="rawdata",
)

# Preprocessed files saved here
corpus_output_data = PipelineData(
    "corpus",
    datastore=datastore,
    output_path_on_compute="corpus",
)

In [5]:
compute_name = "corpus-compute"
vm_size = "STANDARD_D11_V2"
if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('Found compute target: ' + compute_name)
else:
    print('Creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size=vm_size,  # STANDARD_NC6 is GPU-enabled
                                                                min_nodes=0,
                                                                max_nodes=4)
    # create the compute target
    compute_target = ComputeTarget.create(
        ws, compute_name, provisioning_config)

    # Can poll for a minimum number of nodes and for a specific timeout.
    # If no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(
        show_output=True, min_node_count=None, timeout_in_minutes=20)

    # For a more detailed view of current cluster status, use the 'status' property
    print(compute_target.status.serialize())



Found compute target: corpus-compute


In [6]:
env = Environment.from_pip_requirements("sbsdeployment", "./requirements.txt")
runconfig = RunConfiguration()
runconfig.environment = env


In [7]:

process_arguments = ["--input", blob_input_data, "--output", corpus_output_data]
process_step = PythonScriptStep(
    script_name="build_corpus.py",
    arguments=process_arguments,
    inputs=[blob_input_data],
    outputs=[corpus_output_data],
    compute_target=compute_target,
    source_directory=os.getcwd(),
    runconfig=runconfig,
    allow_reuse=True,
)

In [ ]:
predictions = Pipeline(ws, steps=[process_step])
predictions_run = Experiment(ws, "build_corpus").submit(predictions)
predictions_run.wait_for_completion()

In [ ]:

print('done')